In [1]:
import pandas as pd
import nltk as nltk
from emoji import UNICODE_EMOJI
import unicodedata as unicodedata
import re
from __future__ import print_function
import os  # for os.path.basename

import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.manifold import MDS


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
respuestas=pd.read_excel('/Users/pepe_opi/Downloads/mi_salud/results_export_964.xlsx')

In [3]:
respuestas.head()

,Contact UUID,URN,Name,Groups,Response 9 (Category) - uncaught,Response 9 (Value) - uncaught,Response 9 (Text) - uncaught,Response 8 (Category) - uncaught,Response 8 (Value) - uncaught,Response 8 (Text) - uncaught,Response 10 (Category) - uncaught,Response 10 (Value) - uncaught,Response 10 (Text) - uncaught,Response 9 (Category) - uncaught.1,Response 9 (Value) - uncaught.1,Response 9 (Text) - uncaught.1,Response 11 (Category) - uncaught,Response 11 (Value) - uncaught,Response 11 (Text) - uncaught,Response 10 (Category) - uncaught.1,Response 10 (Value) - uncaught.1,Response 10 (Text) - uncaught.1,menu (Category) - uncaught,menu (Value) - uncaught,menu (Text) - uncaught,Response 11 (Category) - uncaught.1,Response 11 (Value) - uncaught.1,Response 11 (Text) - uncaught.1,respuesta_pregs (Category) - uncaught,respuesta_pregs (Value) - uncaught,respuesta_pregs (Text) - uncaught,menu_pd (Category) - uncaught,menu_pd (Value) - uncaught,menu_pd (Text) - uncaught,Response 12 (Category) - uncaught,Response 12 (Value) - uncaught,Response 12 (Text) - uncaught
0,3bf0c4bb-d5db-4e0d-ab29-98a7ee6b2b6a,1104797289620477,NaN,"ALL, PUERPERIUM, PUERPERIUM_MS, PUERPERIUM_MS_...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e7e51622-a160-4bb4-9b7d-785c0c2bfbc7,20999,NaN,NaN,Other,La solicitud de enviar el SMS por Cobrar a 552...,La solicitud de enviar el SMS por Cobrar a 552...,Other,NaN,La solicitud de enviar el SMS por Cobrar a 552...,Other,La solicitud de enviar el SMS por Cobrar a 552...,La solicitud de enviar el SMS por Cobrar a 552...,No,no,Tu mensaje ha excedido el tiempo de respuesta ...,NaN,NaN,NaN,NaN,NaN,NaN,No,no,Tu mensaje ha excedido el tiempo de respuesta ...,NaN,NaN,NaN,NaN,NaN,NaN,seguridad,2,Tienes un SMSxCobrar esperando ser enviado a $...,NaN,NaN,NaN
2,7ea8e04b-e8d0-4628-bf35-350a784d5e2f,+525559652925,pedro vallejo,"ALL, ALTOPD",Emoji,🙅,🙅,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3059e012-6fd3-445d-8da4-5fc36d45f8dc,1326847360726593,NaN,ALTOPD,NaN,NaN,NaN,NaN,NaN,NaN,Other,Ya no quiero recibir mensajes,Ya no quiero recibir mensajes,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42bf4e3f-a396-4eb7-a987-b76cba779e92,+527711780721,Rosy,"ALL, ALTOPD",Other,Hola gordo,Hola gordo,con_nombre,Rosy,Hola gordo,Other,Hola gordo,Hola gordo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
respuestas=respuestas[['Contact UUID', 'URN', 'Name', 'Groups',
       'Response 9 (Category) - uncaught', 'Response 9 (Value) - uncaught',
       'Response 9 (Text) - uncaught']]

respuestas.columns=['id_contacto', 'urn', 'nombre', 'listas', 'uncaught_resp_9_cat',
                    'uncaught_resp_9_val', 'uncaught_resp_9_text']

In [5]:
respuestas.head()

,id_contacto,urn,nombre,listas,uncaught_resp_9_cat,uncaught_resp_9_val,uncaught_resp_9_text
0,3bf0c4bb-d5db-4e0d-ab29-98a7ee6b2b6a,1104797289620477,NaN,"ALL, PUERPERIUM, PUERPERIUM_MS, PUERPERIUM_MS_...",NaN,NaN,NaN
1,e7e51622-a160-4bb4-9b7d-785c0c2bfbc7,20999,NaN,NaN,Other,La solicitud de enviar el SMS por Cobrar a 552...,La solicitud de enviar el SMS por Cobrar a 552...
2,7ea8e04b-e8d0-4628-bf35-350a784d5e2f,+525559652925,pedro vallejo,"ALL, ALTOPD",Emoji,🙅,🙅
3,3059e012-6fd3-445d-8da4-5fc36d45f8dc,1326847360726593,NaN,ALTOPD,NaN,NaN,NaN
4,42bf4e3f-a396-4eb7-a987-b76cba779e92,+527711780721,Rosy,"ALL, ALTOPD",Other,Hola gordo,Hola gordo


In [6]:
respuestas['uncaught_resp_9_cat'].value_counts()

Other            954
Like-fb          452
si               312
Claro            226
no               153
gracias          136
Ok                98
Emoji             56
Buenas noches     39
Gracias           35
Hasta luego        6
Twitter image      5
ok                 1
Name: uncaught_resp_9_cat, dtype: int64

In [7]:
respuestas.loc[respuestas['uncaught_resp_9_cat']=='Twitter image', 'uncaught_resp_9_text'].value_counts()

https://t.co/HouKMudLQc     1
https://t.co/rbsa5QOafg     1
 https://t.co/8kYuMPNM3V    1
https://t.co/az6m6XXjoL     1
https://t.co/Waly9IfMui     1
Name: uncaught_resp_9_text, dtype: int64

In [8]:
respuestas.loc[respuestas['uncaught_resp_9_cat']=='Hasta luego', 'uncaught_resp_9_text'].value_counts()

Hasta luego            2
HASTA LUEGO            1
Hasta luego!           1
Hasta luego gracias    1
hasta luego!           1
Name: uncaught_resp_9_text, dtype: int64

In [9]:
respuestas['uncaught_resp_9_cat'].value_counts()

Other            954
Like-fb          452
si               312
Claro            226
no               153
gracias          136
Ok                98
Emoji             56
Buenas noches     39
Gracias           35
Hasta luego        6
Twitter image      5
ok                 1
Name: uncaught_resp_9_cat, dtype: int64

## Categorías actuales que sí funcionan

Las categorías like de fb, imagen de twitter y hasta luego sí captan sólo a esos tipos de mensajes, por lo que estas reglas deben mantenerse. Por ello, quito estas categorías para la clusterización

In [10]:
respuestas['categ_opi']=None

In [11]:
respuestas['categ_opi'].loc[respuestas['uncaught_resp_9_cat'].isin(list(['Like-fb', 'Twitter image','Hasta luego']))]=respuestas['uncaught_resp_9_cat'].loc[respuestas['uncaught_resp_9_cat'].isin(list(['Like-fb', 'Twitter image','Hasta luego']))]

In [12]:
nulos=respuestas.loc[respuestas.uncaught_resp_9_text.isnull()]
respuestas=respuestas.loc[respuestas.uncaught_resp_9_text.notnull()]

In [13]:
respuestas.head()

,id_contacto,urn,nombre,listas,uncaught_resp_9_cat,uncaught_resp_9_val,uncaught_resp_9_text,categ_opi
1,e7e51622-a160-4bb4-9b7d-785c0c2bfbc7,20999,NaN,NaN,Other,La solicitud de enviar el SMS por Cobrar a 552...,La solicitud de enviar el SMS por Cobrar a 552...,None
2,7ea8e04b-e8d0-4628-bf35-350a784d5e2f,+525559652925,pedro vallejo,"ALL, ALTOPD",Emoji,🙅,🙅,None
4,42bf4e3f-a396-4eb7-a987-b76cba779e92,+527711780721,Rosy,"ALL, ALTOPD",Other,Hola gordo,Hola gordo,None
5,95644138-5bec-48a2-9d87-a18fa01978d7,+525513627389,MiguelVilchis,"ALL, ALTOPD",Gracias,gracias,"Si, gracias",None
6,0ab70bd4-c6b7-4b0a-8440-18db8ebd5bae,1578880975479032,NaN,"ALL, PUERPERIUM, PUERPERIUM_MS, PUERPERIUM_MS_...",no,No,No,None


In [14]:
respuestas['uncaught_resp_9_cat'].value_counts()

Other            953
Like-fb          452
si               312
Claro            226
no               153
gracias          136
Ok                98
Emoji             56
Buenas noches     39
Gracias           35
Hasta luego        6
Twitter image      5
ok                 1
Name: uncaught_resp_9_cat, dtype: int64

## Definir reglas para clasificación de string pre-modelo

### Pasar a minúsculas

In [15]:
respuestas['uncaught_resp_9_text']=respuestas['uncaught_resp_9_text'].str.lower()

 ### Quitar leading y trailing spaces, quitar puntuacion

In [16]:
respuestas['uncaught_resp_9_text']=respuestas['uncaught_resp_9_text'].str.strip()

In [17]:
respuestas['uncaught_resp_9_text_wc'] = respuestas['uncaught_resp_9_text'].apply(lambda x: len(str(x).split(" ")))
respuestas['uncaught_resp_9_text_nchar'] = respuestas['uncaught_resp_9_text'].apply(lambda x: len(str(x)))

In [18]:
respuestas['categ_opi'].loc[(respuestas['uncaught_resp_9_text'].str.count('\\?')==respuestas['uncaught_resp_9_text_nchar'])]='duda'

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
respuestas['uncaught_resp_9_text_mod']=respuestas['uncaught_resp_9_text'].str.replace('[^\w\s]','')

In [20]:
respuestas['uncaught_resp_9_text_mod']=respuestas['uncaught_resp_9_text_mod'].str.strip()

In [21]:
respuestas['uncaught_resp_9_text_mod']

1       la solicitud de enviar el sms por cobrar a 552...
2                                                        
4                                              hola gordo
5                                              si gracias
6                                                      no
12                                                     no
16      httpsscontentxxfbcdnnetvt3919976p100x100851587...
17                                                     no
20      httpsscontentdfw51xxfbcdnnetvt3919976p100x1008...
21                                                     no
22      te llame y no pude localizarte tramita en line...
28                                           hola gracias
29      httpsscontentxxfbcdnnetvt3919976851557_3692392...
32                                                     si
35                                                     si
38                                                     si
45                                                  59508
47            

In [22]:
respuestas['uncaught_resp_9_text_wc'] = respuestas['uncaught_resp_9_text_mod'].apply(lambda x: len(str(x).split(" ")))
respuestas['uncaught_resp_9_text_nchar'] = respuestas['uncaught_resp_9_text_mod'].apply(lambda x: len(str(x)))

### Capturar emojis

Actualmente, si el mensaje contiene un emoji, es canalizado a la categoría Emoji. Este filtro debe ser más estricto, pues usualmente ese emoji es usado en un mensaje real

In [23]:
def is_emoji(s):
    count = 0
    for emoji in range(1, len(UNICODE_EMOJI)):
        count += s.count(list(UNICODE_EMOJI.keys())[emoji])
    return count

In [24]:
es_emoji=pd.DataFrame()
for i in range(0, respuestas.shape[0]):
    es=pd.DataFrame({'es':is_emoji(respuestas['uncaught_resp_9_text'].iloc[i])}, index=[0])
    es_emoji=pd.concat([es_emoji, es], ignore_index=True)

In [25]:
respuestas=respuestas.reset_index(drop=True)

In [26]:
respuestas=pd.concat([respuestas, es_emoji], axis=1)

In [27]:
respuestas.head()

,id_contacto,urn,nombre,listas,uncaught_resp_9_cat,uncaught_resp_9_val,uncaught_resp_9_text,categ_opi,uncaught_resp_9_text_wc,uncaught_resp_9_text_nchar,uncaught_resp_9_text_mod,es
0,e7e51622-a160-4bb4-9b7d-785c0c2bfbc7,20999,NaN,NaN,Other,La solicitud de enviar el SMS por Cobrar a 552...,la solicitud de enviar el sms por cobrar a 552...,None,24,144,la solicitud de enviar el sms por cobrar a 552...,0
1,7ea8e04b-e8d0-4628-bf35-350a784d5e2f,+525559652925,pedro vallejo,"ALL, ALTOPD",Emoji,🙅,🙅,None,1,0,,1
2,42bf4e3f-a396-4eb7-a987-b76cba779e92,+527711780721,Rosy,"ALL, ALTOPD",Other,Hola gordo,hola gordo,None,2,10,hola gordo,0
3,95644138-5bec-48a2-9d87-a18fa01978d7,+525513627389,MiguelVilchis,"ALL, ALTOPD",Gracias,gracias,"si, gracias",None,2,10,si gracias,0
4,0ab70bd4-c6b7-4b0a-8440-18db8ebd5bae,1578880975479032,NaN,"ALL, PUERPERIUM, PUERPERIUM_MS, PUERPERIUM_MS_...",no,No,no,None,1,2,no,0


In [28]:
respuestas['categ_opi'].loc[(respuestas.es>0) &
                            (respuestas.uncaught_resp_9_text_mod=='')]='emoji'

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


### Quitar emojis que quedan en el texto

In [29]:
def give_emoji_free_text(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

In [30]:
for i in range(0, respuestas.shape[0]):
    respuestas['uncaught_resp_9_text_mod'].iloc[i]=give_emoji_free_text(respuestas.uncaught_resp_9_text_mod.iloc[i])

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


### Quitar  acentos

In [31]:
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('https')) &
              (respuestas.uncaught_resp_9_text_wc==1) & (respuestas.categ_opi.isnull())]='spam'



/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [32]:
for i in range(0, respuestas.shape[0]):
    respuestas['uncaught_resp_9_text_mod'].iloc[i]=unicodedata.normalize('NFD', respuestas['uncaught_resp_9_text_mod'].iloc[i]).encode('ascii', 'ignore').decode('utf-8')

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [34]:
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('si')) &
                            (respuestas.uncaught_resp_9_text_nchar==2)]='si'

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [35]:
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('no')) &
                            (respuestas.uncaught_resp_9_text_nchar==2)]='no'

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [36]:
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('gracias')) &
               (respuestas.uncaught_resp_9_text_wc==1) ]='gracias'

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [37]:
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('ok')) & (respuestas.uncaught_resp_9_text_wc==1) &
              (respuestas['categ_opi'].isnull())]='ok'

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


### Quitar stopwords

In [38]:
stop=nltk.corpus.stopwords.words("spanish")



In [39]:
for i in range(0, len(stop)):
    stop[i]=unicodedata.normalize('NFD', stop[i]).encode('ascii', 'ignore').decode('utf-8')

In [40]:

respuestas['uncaught_resp_9_text_mod']=respuestas['uncaught_resp_9_text_mod'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [41]:
respuestas.loc[(respuestas['uncaught_resp_9_text_mod']=='') &
              respuestas['categ_opi'].isnull()]

#respuestas=respuestas.loc[~respuestas.index.isin(fallas.index)]



,id_contacto,urn,nombre,listas,uncaught_resp_9_cat,uncaught_resp_9_val,uncaught_resp_9_text,categ_opi,uncaught_resp_9_text_wc,uncaught_resp_9_text_nchar,uncaught_resp_9_text_mod,es
207,60602997-0fc6-4765-8a92-1c2c92101b35,1637367463016051,Brenda Anaya,"ALL, PUERPERIUM, PUERPERIUM_MS, PUERPERIUM_MS_...",Emoji,Todos 😊,todos 😊,None,1,5,,1
307,422328aa-bc22-4907-9a84-e5646a0346e2,+522761152605,NaN,"ALL, ALTOPD",Other,Quien Eres...?,quien eres...?,None,2,10,,0
316,736bfad0-71ad-4587-b0c7-db6085238004,1657679220981102,Monse Torres,"PUERPERIUM, PUERPERIUM_MS, PUERPERIUM_MS_ow, P...",Other,Algo,algo,None,1,4,,0
810,5f6b7a3a-fb1a-42c5-930b-1a5381ef99f9,1323614167740738,PaTty Coronel,"ALL, PERSONAL_SALUD, PERSONAL_SALUD_ow, PERSON...",Other,Antes que nada,antes que nada,None,3,14,,0
829,b2c26540-e501-402c-998a-83195254faf2,1808051242549508,Faber Hernandez,NaN,Other,.,.,None,1,0,,0
1103,831a3f24-ddc4-470f-b849-26655879ec81,martinfsotoj,Monitor Científico,NaN,Other,O,o,None,1,1,,0
1741,5d794ccf-1d19-4063-8b8e-d402bed6e268,1885331878178808,James Scott Sanders,NaN,Other,...,...,None,1,0,,0
1880,7ece8aac-93ab-4462-90f2-b63a335d6bd1,1835399329831739,Nancy Herrera,"CR-Grupo-DM, CR-Todos",Other,A,a,None,1,1,,0
2274,5faebf12-3b9d-400c-8283-a5665972fe46,1600903953354258,Mary Guevara,"CR-Grupo-DM, CR-Todos",Other,Eso lo sé,eso lo sé,None,3,9,,0


In [42]:
from nltk.stem.snowball import SnowballStemmer

In [43]:
mensajes=respuestas['uncaught_resp_9_text_mod'].loc[respuestas.categ_opi.isnull()]

In [44]:
stemmer = SnowballStemmer("spanish")

In [45]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [46]:
totalvocab_stemmed = []
totalvocab_tokenized = []
lista_fin=list()
for i in mensajes:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)


In [47]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print( 'there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 13780 items in vocab_frame


In [48]:
vocab_frame.index.value_counts()

buen                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          455
hol                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           292
pued

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=200000,
                                 min_df=0,
                                 use_idf=True, tokenizer=tokenize_and_stem,
                                 ngram_range=(1,2))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(mensajes) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 963 ms, sys: 21.7 ms, total: 985 ms
Wall time: 1.01 s
(1270, 13034)


In [50]:
tfidf_matrix

<1270x13034 sparse matrix of type '<class 'numpy.float64'>'
	with 25238 stored elements in Compressed Sparse Row format>

In [51]:
terms = tfidf_vectorizer.get_feature_names()

In [52]:
terms

['11al',
 '11al marz',
 '14abr18',
 '14abr18 info',
 '173076059407762274secretari',
 '173076059407762274secretari salud',
 '1an0',
 '1eraseccioncentr',
 '1eraseccioncentr tabasc',
 '1l',
 '1l primer',
 '1o',
 '1o derechohabient',
 '1temgesic',
 '1temgesic 2cymbalt',
 '24hrs',
 '24hrs cuent',
 '26an',
 '26an medic',
 '2cymbalt',
 '2cymbalt cad',
 '2pes',
 '2pes civ',
 '300sms',
 '3700c',
 '3700c coloni',
 '3a',
 '3a cda',
 '4mes',
 '4mes embaraz',
 '4to',
 '4to pis',
 '59108httpswwwgobmxsaludcnegsrarticulosjornadasintensivasdevasectomiasinbisturienlasecretariadesaludidiom',
 '5de',
 '5de may',
 '5m',
 '5m lleg',
 '679025138919951274jos',
 '679025138919951274jos narr',
 '7an',
 '7an emo',
 '8bmes',
 '8bmes apen',
 'aap',
 'aap httpswwwfacebookcomasociacionmexicanadepediatriavideos633024427039692',
 'abaj',
 'abaj dos',
 'abarrot',
 'abastec',
 'abastec vacun',
 'abat',
 'abat pil',
 'abiert',
 'abiert bols',
 'abiert call',
 'abiert cdjuarez',
 'abiert medic',
 'abiert vecin',
 'abog',
 

In [53]:

from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)
print()


In [54]:
dist.shape

(1270, 1270)

In [55]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 1.48 s, sys: 46.5 ms, total: 1.53 s
Wall time: 1.61 s


In [56]:
pd.DataFrame(clusters)[0].value_counts()

2    1066
3      86
4      49
1      44
0      25
Name: 0, dtype: int64

In [57]:
df = { 'id_contacto': respuestas.id_contacto.loc[respuestas.categ_opi.isnull()],
         'urn': respuestas.id_contacto.loc[respuestas.categ_opi.isnull()],
         'nombre': respuestas.nombre.loc[respuestas.categ_opi.isnull()],
         'texto': respuestas.uncaught_resp_9_text.loc[respuestas.categ_opi.isnull()],
         'texto_mod': respuestas.uncaught_resp_9_text_mod.loc[respuestas.categ_opi.isnull()],
         'cat': respuestas.uncaught_resp_9_cat.loc[respuestas.categ_opi.isnull()],
         'cluster': clusters,
         'wc':respuestas.uncaught_resp_9_text_wc.loc[respuestas.categ_opi.isnull()]}

frame = pd.DataFrame(df)

In [58]:
frame=frame.sort_values(['cluster'])
frame[['cluster', 'texto', 'texto_mod', 'wc']]

,cluster,texto,texto_mod,wc
236,0,muchas gracias,muchas gracias,2
119,0,gracias. saludos.,gracias saludos,2
115,0,ok gracias,ok gracias,2
1030,0,ok gracias,ok gracias,2
232,0,muchas gracias 😉😉,muchas gracias,2
2326,0,ok gracias,ok gracias,2
1414,0,ok gracias,ok gracias,2
1661,0,muchas gracias,muchas gracias,2
43,0,si gracias,gracias,2
1739,0,muchas gracias,muchas gracias,2


In [59]:
print(frame.groupby('cluster').agg({'wc':'mean'}) )
print(frame.groupby('cluster').agg({'wc':'std'}) )

#print(grouped.mean() )
#print(grouped.std() )


                wc
cluster           
0         2.320000
1         2.636364
2        23.286116
3         6.000000
4         1.020408
                wc
cluster           
0         0.988264
1         0.613450
2        26.561113
3         0.000000
4         0.142857


In [60]:


print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :10]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0], end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d titles:" % i, end='')
    for title in frame.loc[frame.cluster==i, 'texto'].head(5):
        print(' %s,' % title, end='')
    print() #add whitespace
    print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 words: gracias, ok, ok, muchas, muchas, gracias, saludos, ok, atencion, ok,

Cluster 0 titles: muchas gracias, gracias. saludos., ok gracias, ok gracias, muchas gracias 😉😉,

Cluster 1 words: buenas, tarde, buenas, hola, hola, epatitis, eps, eps, eps, eps,

Cluster 1 titles: hola buenas tardes, hola buena tarde, buenas tardes, hola, buenas tardes., hola buenas tardes,

Cluster 2 words: buenas, puedo, hola, vacunas, saludos, dia, dias

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


, buenas, tarde, buenas,

Cluster 2 titles: si tengo problema de sobrepeso, mido 1.60, peso 78 kg, galería de ideas added a new photo.
https://www.facebook.com/galeriadeideas/posts/756764904521766, buenos días las vacunas también ahí en el imss, reminder - by registering (free) with https://t.co/c3xgbpvqpu  your account will be credited with 50 free job adverts., no lo disfrute, por eso me voy,

Cluster 3 words: numero, telefono, numero, zumba, epatitis, eps, eps, eps, eps, eps,

Cluster 3 titles: ¿cuál es el número de teléfono?, ¿cuál es el número de teléfono?, ¿cuál es el número de teléfono?, ¿cuál es el número de teléfono?, ¿cuál es el número de teléfono?,

Cluster 4 words: hola, hola, envoyer, eps, eps, eps, eps, eps, epidemia, epidemia,

Cluster 4 titles: hola, hola, hola, hola, hola,





In [61]:


MDS()

# convert two components as we're plotting points in a two-dimensional plane
# "precomputed" because we provide a distance matrix
# we will also specify `random_state` so the plot is reproducible.
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)

pos = mds.fit_transform(dist)  # shape (n_components, n_samples)

xs, ys = pos[:, 0], pos[:, 1]
print()
print()

In [63]:
#set up colors per clusters using a dict
cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a', 4: '#66a61e'}

#set up cluster names using a dict
cluster_names = {0: 'gracias', 
                 1: 'buenas', 
                 2: 'preguntas vacunas', 
                 3: 'Numero telefono', 
                 4: 'Hola'}

In [ ]:
#some ipython magic to show the matplotlib plots inline
%matplotlib inline 

#create data frame that has the result of the MDS plus the cluster numbers and titles
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=frame.texto)) 

#group by cluster
groups = df.groupby('label')


# set up plot
fig, ax = plt.subplots(figsize=(17, 9)) # set size
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

#iterate through groups to layer the plot
#note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, 
            label=cluster_names[name], color=cluster_colors[name], 
            mec='none')
    ax.set_aspect('auto')
    ax.tick_params(\
        axis= 'x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
    ax.tick_params(\
        axis= 'y',         # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelleft='off')
    
ax.legend(numpoints=1)  #show legend with only 1 point

#add label in x,y position with the label as the film title
for i in range(len(df)):
    ax.text(df.iloc[i]['x'], df.iloc[i]['y'], df.iloc[i]['title'], size=8)  

    
    
#plt.show() #show the plot

#uncomment the below to save the plot if need be
#plt.savefig('clusters_small_noaxes.png', dpi=200)

In [ ]:
plt.close()


In [64]:
import mpld3

In [65]:
#define custom toolbar location
class TopToolbar(mpld3.plugins.PluginBase):
    """Plugin for moving toolbar to top of figure"""

    JAVASCRIPT = """
    mpld3.register_plugin("toptoolbar", TopToolbar);
    TopToolbar.prototype = Object.create(mpld3.Plugin.prototype);
    TopToolbar.prototype.constructor = TopToolbar;
    function TopToolbar(fig, props){
        mpld3.Plugin.call(this, fig, props);
    };

    TopToolbar.prototype.draw = function(){
      // the toolbar svg doesn't exist
      // yet, so first draw it
      this.fig.toolbar.draw();

      // then change the y position to be
      // at the top of the figure
      this.fig.toolbar.toolbar.attr("x", 150);
      this.fig.toolbar.toolbar.attr("y", 400);

      // then remove the draw function,
      // so that it is not called again
      this.fig.toolbar.draw = function() {}
    }
    """
    def __init__(self):
        self.dict_ = {"type": "toptoolbar"}

In [66]:
fig=plt.figure(figsize=(20, 20), dpi= 200, facecolor='w', edgecolor='k')
fig

In [67]:
#create data frame that has the result of the MDS plus the cluster numbers and titles
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=mensajes)) 

#group by cluster
groups = df.groupby('label')

#define custom css to format the font and to remove the axis labeling
css = """
text.mpld3-text, div.mpld3-tooltip {
  font-family:Arial, Helvetica, sans-serif;
}

g.mpld3-xaxis, g.mpld3-yaxis {
display: none; }

svg.mpld3-figure {
margin-left: -200px;}
"""

# Plot 
fig, ax = plt.subplots(figsize=(15,15)) #set plot size
#ax.margins(0.03) # Optional, just adds 5% padding to the autoscaling

#iterate through groups to layer the plot
#note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
for name, group in groups:
    points = ax.plot(group.x, group.y, marker='o', linestyle='', ms=18, 
                     label=cluster_names[name], mec='none', 
                     color=cluster_colors[name])
    #ax.set_aspect('auto')
    labels = [i for i in group.title]
    
    #set tooltip using points, labels and the already defined 'css'
    tooltip = mpld3.plugins.PointHTMLTooltip(points[0], labels,
                                       voffset=10, hoffset=10, css=css)
    #connect tooltip to fig
    mpld3.plugins.connect(fig, tooltip, TopToolbar())    
    
    #set tick marks as blank
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticks([])
    
    #set axis as blank
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)

    
ax.legend(numpoints=1) #show legend with only one dot

mpld3.display() #show the plot

In [ ]:
respuestas.uncaught_resp_9_text_wc

In [70]:
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('hola')) &
              (respuestas.uncaught_resp_9_text_wc<=2) &
              (respuestas.categ_opi.isnull())]='hola'


/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [74]:
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('hola|buenas tardes|buenas noches|buenos dias')) &
              (respuestas.uncaught_resp_9_text_wc<=4) &
              (respuestas.categ_opi.isnull())]='hola'


/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [81]:
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('gracias')) &
              (respuestas.uncaught_resp_9_text_wc<=5) &
              (respuestas.categ_opi.isnull())]='gracias'


/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [84]:
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('horario')) &
               (respuestas.uncaught_resp_9_text_mod.str.contains('atencion')) &
              (respuestas.categ_opi.isnull())]='pregunta_horario'


/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [88]:
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('numero')) &
               (respuestas.uncaught_resp_9_text_mod.str.contains('telefono')) &
               (respuestas.uncaught_resp_9_text_wc<10) &
              (respuestas.categ_opi.isnull())]='pregunta_telefono'



/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## Volver a correr clusters sin basura

In [89]:
mensajes=respuestas['uncaught_resp_9_text_mod'].loc[respuestas.categ_opi.isnull()]

In [90]:
totalvocab_stemmed = []
totalvocab_tokenized = []
lista_fin=list()
for i in mensajes:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [91]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print( 'there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 13287 items in vocab_frame


In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=200000,
                                 min_df=0,
                                 use_idf=True, tokenizer=tokenize_and_stem,
                                 ngram_range=(1,2))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(mensajes) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 881 ms, sys: 19.7 ms, total: 901 ms
Wall time: 920 ms
(1021, 13002)


In [93]:
terms = tfidf_vectorizer.get_feature_names()

In [94]:
dist = 1 - cosine_similarity(tfidf_matrix)

In [95]:
from sklearn.cluster import KMeans

num_clusters = 4

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 1.71 s, sys: 35.8 ms, total: 1.75 s
Wall time: 1.81 s


In [96]:
pd.DataFrame(clusters)[0].value_counts()

1    654
2    237
3     85
0     45
Name: 0, dtype: int64

In [97]:
df = { 'id_contacto': respuestas.id_contacto.loc[respuestas.categ_opi.isnull()],
         'urn': respuestas.id_contacto.loc[respuestas.categ_opi.isnull()],
         'nombre': respuestas.nombre.loc[respuestas.categ_opi.isnull()],
         'texto': respuestas.uncaught_resp_9_text.loc[respuestas.categ_opi.isnull()],
         'texto_mod': respuestas.uncaught_resp_9_text_mod.loc[respuestas.categ_opi.isnull()],
         'cat': respuestas.uncaught_resp_9_cat.loc[respuestas.categ_opi.isnull()],
         'cluster': clusters,
         'wc':respuestas.uncaught_resp_9_text_wc.loc[respuestas.categ_opi.isnull()]}

frame = pd.DataFrame(df)

In [98]:
frame.sort_values('cluster')

,cat,cluster,id_contacto,nombre,texto,texto_mod,urn,wc
1557,Other,0,d63a8471-f32e-463a-8593-0f2d4df78323,Jose Joaquin Agudelo Belalcazar,carolina rodriguez added 3 new photos.\nhttps:...,carolina rodriguez added 3 new photos httpswww...,d63a8471-f32e-463a-8593-0f2d4df78323,6
1623,Other,0,b8abb4e6-9368-432c-8724-ba5781616fdd,Zareth Cortes,mi bebe,bebe,b8abb4e6-9368-432c-8724-ba5781616fdd,2
2401,Other,0,423db0ef-8e4f-403d-8d2d-b9f17d6217d6,Samuel Alonso Razo Loza,desdeelnorte added a new photo.\nhttps://www.f...,desdeelnorte added new photo httpswwwfacebookc...,423db0ef-8e4f-403d-8d2d-b9f17d6217d6,5
2400,Other,0,8acb2cba-21e6-4550-a100-3f60e28d0409,Ramona Gutierrez Mena,idea co added a new video: el nevado de toluca...,idea co added new video nevado toluca mejores ...,8acb2cba-21e6-4550-a100-3f60e28d0409,14
2396,Other,0,73e51b85-0a3f-423e-8b66-a2e701b651f1,Itzel Plascencia,guardia nocturna added a new video: guardia no...,guardia nocturna added new video guardia noctu...,73e51b85-0a3f-423e-8b66-a2e701b651f1,11
949,Other,0,e3650594-c024-4fe5-a484-ba2539dfb75a,Arbey Cruz,en la mira added a new photo.\nhttps://www.fac...,mira added new photo httpswwwfacebookcompermal...,e3650594-c024-4fe5-a484-ba2539dfb75a,7
2159,Other,0,09491b2b-1444-4dc9-a0cc-89f1642bf2a2,Paula Sirius,imagen noticias added a new video: enfermera a...,imagen noticias added new video enfermera agre...,09491b2b-1444-4dc9-a0cc-89f1642bf2a2,16
196,si,0,151924e3-a15c-499d-8d7b-baeeebabbe5f,NaN,si quieres pagarle a este usuario su proximo m...,quieres pagarle usuario proximo mensaje solo r...,151924e3-a15c-499d-8d7b-baeeebabbe5f,22
198,Other,0,4f642e88-9e46-447b-aef3-713f01bf0ab2,Gama Gamiux Gamiux,mi bebe ya nació,bebe nacio,4f642e88-9e46-447b-aef3-713f01bf0ab2,4
2158,Other,0,f44b53e1-0d0d-42bf-aa18-33b69111b1d4,Paula Sirius,imagen noticias added a new video: enfermera a...,imagen noticias added new video enfermera agre...,f44b53e1-0d0d-42bf-aa18-33b69111b1d4,16


In [99]:
print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :10]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0], end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d titles:" % i, end='')
    for title in frame.loc[frame.cluster==i, 'texto'].head(5):
        print(' %s,' % title, end='')
    print() #add whitespace
    print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 words: bebe, added, new, added, photos, new, video, new, phone, is,

Cluster 0 titles: mi bebe, mi bebé esta deshidratado, ¿por que mi bebe estornuda mucho?, mi bebe, todo tecate se enteró por faceboook added a new video: mira lo que dijo de los mexicanos en usa ricardo anaya.
https://www.facebook.com/yenteratedemas/videos/974337659381823/,

Cluster 1 words: puedo, salud, hola, hace, saber, trabajadora, buen, quiero, informacion, pregunta,

Cluster 1 titles: la solicitud de enviar el sms por cobrar a 5521817435 ha expirado ya que excedio el tiempo de respuesta permitido. por favor intentalo nuevamente., te llame y no pude localizarte. tramita en linea tu tarjeta y recibe 4,000 mxn en monedero de liverpool. clic aqui bit.ly/2iucydn vig 03/06/18, 59508, hola si ya cada integrante tenemos fotos con la bb seguido nos tomamos fotos gracias x los consejos, el usuario 2471032511 te envio un mensaje pero no tiene saldo. responde si para recibirlo. precio $2.50

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


In [100]:


MDS()

# convert two components as we're plotting points in a two-dimensional plane
# "precomputed" because we provide a distance matrix
# we will also specify `random_state` so the plot is reproducible.
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)

pos = mds.fit_transform(dist)  # shape (n_components, n_samples)

xs, ys = pos[:, 0], pos[:, 1]
print()
print()

In [102]:
#set up colors per clusters using a dict
cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a'}

#set up cluster names using a dict
cluster_names = {0: '1', 
                 1: '2', 
                 2: '3', 
                 3: '4'}


#create data frame that has the result of the MDS plus the cluster numbers and titles
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=mensajes)) 

#group by cluster
groups = df.groupby('label')

#define custom css to format the font and to remove the axis labeling
css = """
text.mpld3-text, div.mpld3-tooltip {
  font-family:Arial, Helvetica, sans-serif;
}

g.mpld3-xaxis, g.mpld3-yaxis {
display: none; }

svg.mpld3-figure {
margin-left: -200px;}
"""

# Plot 
fig, ax = plt.subplots(figsize=(15,15)) #set plot size
#ax.margins(0.03) # Optional, just adds 5% padding to the autoscaling

#iterate through groups to layer the plot
#note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
for name, group in groups:
    points = ax.plot(group.x, group.y, marker='o', linestyle='', ms=18, 
                     label=cluster_names[name], mec='none', 
                     color=cluster_colors[name])
    #ax.set_aspect('auto')
    labels = [i for i in group.title]
    
    #set tooltip using points, labels and the already defined 'css'
    tooltip = mpld3.plugins.PointHTMLTooltip(points[0], labels,
                                       voffset=10, hoffset=10, css=css)
    #connect tooltip to fig
    mpld3.plugins.connect(fig, tooltip, TopToolbar())    
    
    #set tick marks as blank
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticks([])
    
    #set axis as blank
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)

    
ax.legend(numpoints=1) #show legend with only one dot

mpld3.display() #show the plot

In [143]:
respuestas.shape

(1011, 11)

In [105]:
respuestas.categ_opi.loc[respuestas.uncaught_resp_9_text_mod.str.contains('added') &
              respuestas.categ_opi.isnull()]='spam'


/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## Última clusterización eliminando basura

In [106]:
mensajes=respuestas['uncaught_resp_9_text_mod'].loc[respuestas.categ_opi.isnull()]

In [107]:
totalvocab_stemmed = []
totalvocab_tokenized = []
lista_fin=list()
for i in mensajes:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)
    
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print( 'there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=200000,
                                 min_df=0,
                                 use_idf=True, tokenizer=tokenize_and_stem,
                                 ngram_range=(1,2))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(mensajes) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

terms = tfidf_vectorizer.get_feature_names()

dist = 1 - cosine_similarity(tfidf_matrix)

there are 13061 items in vocab_frame
CPU times: user 860 ms, sys: 16.3 ms, total: 876 ms
Wall time: 893 ms
(993, 12799)


In [108]:
from sklearn.cluster import KMeans

num_clusters = 3

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

pd.DataFrame(clusters)[0].value_counts()

CPU times: user 2.07 s, sys: 41.3 ms, total: 2.11 s
Wall time: 2.17 s


2    646
1    257
0     90
Name: 0, dtype: int64

In [109]:
df = { 'id_contacto': respuestas.id_contacto.loc[respuestas.categ_opi.isnull()],
         'urn': respuestas.id_contacto.loc[respuestas.categ_opi.isnull()],
         'nombre': respuestas.nombre.loc[respuestas.categ_opi.isnull()],
         'texto': respuestas.uncaught_resp_9_text.loc[respuestas.categ_opi.isnull()],
         'texto_mod': respuestas.uncaught_resp_9_text_mod.loc[respuestas.categ_opi.isnull()],
         'cat': respuestas.uncaught_resp_9_cat.loc[respuestas.categ_opi.isnull()],
         'cluster': clusters,
         'wc':respuestas.uncaught_resp_9_text_wc.loc[respuestas.categ_opi.isnull()]}

frame = pd.DataFrame(df)

In [111]:
print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :10]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0], end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d titles:" % i, end='')
    for title in frame.loc[frame.cluster==i, 'texto'].head(5):
        print(' %s,' % title, end='')
    print() #add whitespace
    print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 words: vacunas, hepatitis, vacunas, b, salud, hepatitis, bebe, buen, centro, centro,

Cluster 0 titles: mi bebe, mi bebe, no hay vacunas desde hace dos meses, 2 vacuna, no hay vacuna hepatitis b,

Cluster 1 words: buen, buen, tarde, dia, buen, hola, hola, puedo, saber, dias,

Cluster 1 titles: buen dia, buen dia alguna propuesta en que me puedan apoyar?, buenas noches,
disculpen tendrán algún calendario de festividades de salud que se celebran este año. 
me urge conseguirlo, para que se puedan programar diversas campañas. de antemano gracias, les dejo mi e-mail.
psic_anamar@hotmail.com, buen día quisiera hacer una pregu ta, buen día, esperando que todo marche por buen camino le hago la siguiente pregunta: es necesario vacunar a los niños con un refuerzo contra el sarampión o solamente siguiendo el esquema de la cartilla?? le agradezco mucho la información que me pueda proporcionar,

Cluster 2 words: puedo, hola, salud, hace, quiero, trabajadora, inform

Salen los mismos clusters anteriores, si le subo un cluster, me detecta una cadena que se envió muchas veces. Ya no se puede avanzar más con reglas estáticas

In [115]:
respuestas[['uncaught_resp_9_cat', 'uncaught_resp_9_text', 'categ_opi']]

,uncaught_resp_9_cat,uncaught_resp_9_text,categ_opi
0,Other,la solicitud de enviar el sms por cobrar a 552...,None
1,Emoji,🙅,emoji
2,Other,hola gordo,hola
3,Gracias,"si, gracias",gracias
4,no,no,no
5,no,no,no
6,Like-fb,https://scontent.xx.fbcdn.net/v/t39.1997-6/p10...,Like-fb
7,no,no,no
8,Like-fb,https://scontent-dfw5-1.xx.fbcdn.net/v/t39.199...,Like-fb
9,no,no,no


In [123]:
respuestas.categ_opi.loc[respuestas.categ_opi.isnull()]='Other'

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [125]:
pd.crosstab(respuestas.uncaught_resp_9_cat,  respuestas.categ_opi)

categ_opi,Hasta luego,Like-fb,Other,Twitter image,duda,emoji,gracias,hola,no,ok,pregunta_horario,pregunta_telefono,si,spam
uncaught_resp_9_cat,,,,,,,,,,,,,,
Buenas noches,0,0,33,0,0,0,0,6,0,0,0,0,0,0
Claro,0,0,141,0,0,0,29,0,11,9,0,0,35,1
Emoji,0,0,16,0,0,34,5,0,0,1,0,0,0,0
Gracias,0,0,19,0,0,0,16,0,0,0,0,0,0,0
Hasta luego,6,0,0,0,0,0,0,0,0,0,0,0,0,0
Like-fb,0,452,0,0,0,0,0,0,0,0,0,0,0,0
Ok,0,0,3,0,0,0,10,0,0,85,0,0,0,0
Other,0,0,675,0,4,0,5,113,0,0,9,87,2,58
Twitter image,0,0,0,5,0,0,0,0,0,0,0,0,0,0


In [135]:
respuestas.uncaught_resp_9_cat.value_counts()

Other            953
Like-fb          452
si               312
Claro            226
no               153
gracias          136
Ok                98
Emoji             56
Buenas noches     39
Gracias           35
Hasta luego        6
Twitter image      5
ok                 1
Name: uncaught_resp_9_cat, dtype: int64

In [136]:
respuestas.categ_opi.value_counts()

Other                993
Like-fb              452
si                   309
gracias              185
hola                 120
no                   114
ok                    95
pregunta_telefono     87
spam                  59
emoji                 34
pregunta_horario       9
Hasta luego            6
Twitter image          5
duda                   4
Name: categ_opi, dtype: int64

In [138]:
respuestas.uncaught_resp_9_text.loc[respuestas.categ_opi=='Other']

0       la solicitud de enviar el sms por cobrar a 552...
10      te llame y no pude localizarte. tramita en lin...
16                                                  59508
17                                                mi bebe
29      hola si ya cada integrante tenemos fotos con l...
33      el usuario 2471032511 te envio un mensaje pero...
36      te llame y no pude localizarte. ¡ultima oportu...
37      te llame y no pude localizarte. 3,000 mxn en m...
38                              mi bebé esta deshidratado
39                                 mensaje fuera de flujo
40                                         en 5m llegamos
41      tienes saldo congelado, recuperalo recargando ...
42                      ¿por que mi bebe estornuda mucho?
56                                          si ya me dijo
64                                                mi bebe
71                                 ya no estoy embarazada
72                                           ya no quiero
85            

In [132]:
for i in range(1, 5):
    muestra=respuestas.sample(50)
    muestra.to_csv('/Users/pepe_opi/Documents/GIT/mi-salud/datos/muestra_etiquetado_'+str(i)+'.csv')
#'/Users/pepe_opi/Documents/GIT/mi-salud'

In [415]:
otros=respuestas.uncaught_resp_9_text_mod.loc[respuestas.categ_opi=='Other']
otros=otros.reset_index(drop=True)

In [416]:
otros=otros.str.replace('hola|buenos dias|buenos dia|buen dia|buenas noches|buenas tardes|buena tarde|ola', '')

In [417]:
import numpy as np  # a conventional alias
import sklearn.feature_extraction.text as text

vectorizer = text.CountVectorizer(min_df=1)

dtm = vectorizer.fit_transform(otros)

vocab = np.array(vectorizer.get_feature_names())
#
dtm.shape
#
#
len(vocab)

4611

In [418]:
from sklearn import decomposition

num_topics = 10

num_top_words = 20

clf = decomposition.NMF(n_components=num_topics, random_state=1)


In [419]:
doctopic = clf.fit_transform(dtm)

In [420]:
doctopic

array([[ 0.        ,  0.05516905,  0.        , ...,  0.        ,
         0.00857553,  0.00809495],
       [ 0.        ,  0.        ,  0.00117119, ...,  0.00026126,
         0.        ,  0.0092122 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.05309606,  0.0002313 , ...,  0.0048531 ,
         0.01628378,  0.05123091],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.00707119,  0.        ],
       [ 0.05555095,  0.04320662,  0.        , ...,  0.00635111,
         0.00944195,  0.06244863]])

In [421]:
topic_words = []

for topic in clf.components_:
    word_idx = np.argsort(topic)[::-1][0:num_top_words]
    topic_words.append([vocab[i] for i in word_idx])
    


In [422]:
#doctopic = doctopic / np.sum(doctopic, axis=1, keepdims=True)

In [423]:
len(doctopic)

993

In [424]:
otros[11]

'saldo congelado recuperalo recargando 20 ademas obten 100 mb seguir conectado redes sociales 14abr18 info cac telcel'

In [425]:
scores_mensajes=pd.DataFrame()
for i in range(0, len(doctopic)):
    temas_score=pd.DataFrame(data= doctopic[i])
    scores_mensajes=pd.concat([scores_mensajes, temas_score], axis=1, ignore_index=True)

In [426]:
otros=pd.concat([otros,  scores_mensajes.idxmax(axis=0)], axis=1, ignore_index=True)

In [427]:
otros.columns=['mensaje', 'clase']

In [428]:
otros.clase.value_counts(sort=False)

0    129
1    137
2     10
3     81
4    112
5    117
6     18
7    101
8    130
9    158
Name: clase, dtype: int64

In [434]:
for t in range(len(topic_words)):
    print("Topic {}: {}".format(t, ' '.join(topic_words[t][:15])))

Topic 0: bacteria analisis informacion productos puede alpura personas secretaria marca debe salmonella informa producto familiar pasar
Topic 1: gracias informacion podrian muchas favor numero atencion espero respuesta algun vacunacion informar ustedes situacion podria
Topic 2: of to and for the on bishop suggest from good members add congress names given
Topic 3: hospital general cirugia puedan medico alta personal aqui especialidad atencion mexico dr nombre alla asi
Topic 4: salud secretaria centro dia nombre hoy guias servicio anos ser poder apoyo sector ustedes usted
Topic 5: anos hace mensaje seguro popular san favor drenaje vecinos tres cambiar mexico calle medio direccion
Topic 6: livro um do ser jornal sou nacional aqui queria da medicina codigo foto objetivo certificado
Topic 7: vacuna hepatitis vacunas bebe centro conseguir meses dicen centros sarampion fecha encontrar hijo agotada sabin
Topic 8: saber quisiera gustaria trabajo seguro consulta quiero popular algun alguna tene

In [450]:
otros['mensaje'].loc[otros.clase==0]

2                                                  59508
10                                           5m llegamos
17                                                lizzie
23                                          heidi monroy
24                                                      
36                                                      
46                                                     n
53                                                      
56                                                      
58     podremos encontrar personas interesadas escuch...
64     puede atender dano ambiental foco infecciones ...
82                                        podran agregar
88                                              gracaias
92                                                      
101                                               aversi
102                                           desayunado
118                                    causas tabaquismo
139                            

## Temas encontrados

0. Spam (fue un error)
1. Info administrativa
2. Mensajes en inglés (fue un error)
3. Preguntas de doctores u hospitales
4. Preguntas de programas de SS
5. Denuncias
6. Spam (fue un error)
7. Orientación médica (o sobre vacunas)
8. Solicitud de trabajo
9. Todas las anteriores